### Loading Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
pd.options.display.max_columns=10000 
%matplotlib inline

In [2]:
users = pd.read_csv('takehome_users.csv', encoding='latin-1')
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [4]:
df = pd.read_csv('takehome_user_engagement.csv')
df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


### Cleaning, transfomation

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [6]:
df['time_stamp'] = pd.to_datetime(df.time_stamp)
df = df.sort_values('time_stamp')
df.head()

,time_stamp,user_id,visited
178140,2012-05-31 08:20:06,10012,1
59486,2012-05-31 15:47:36,3428,1
175638,2012-05-31 17:19:37,9899,1
26821,2012-05-31 21:58:33,1693,1
109716,2012-06-01 00:17:30,6102,1


In [7]:
df= df.set_index('time_stamp').groupby('user_id')['user_id'].rolling('7D').count().rename('count_7_days')\
.reset_index()


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,207917.0,5913.314197,3394.941674,1.0,3087.0,5682.0,8944.0,12000.0
count_7_days,207917.0,4.381393,2.042666,1.0,3.0,4.0,6.0,7.0


In [9]:
df = df.groupby('user_id')['count_7_days'].max().reset_index()
df.head()

,user_id,count_7_days
0,1,1.0
1,2,3.0
2,3,1.0
3,4,1.0
4,5,1.0


In [10]:
df['adopted_user'] = df['count_7_days'].apply(lambda x: 1 if x>=3 else 0) 
df.head()

,user_id,count_7_days,adopted_user
0,1,1.0,0
1,2,3.0,1
2,3,1.0,0
3,4,1.0,0
4,5,1.0,0


In [11]:
df.adopted_user.value_counts()

0    7221
1    1602
Name: adopted_user, dtype: int64

In [12]:
df.drop('count_7_days', axis=1, inplace=True)

In [13]:
df.rename(columns={'user_id': 'object_id'}, inplace=True)
df.head()

,object_id,adopted_user
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [14]:
df_users= pd.merge(users, df, on='object_id', how='left')
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0


In [15]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
 10  adopted_user                8823 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 1.1+ MB


In [16]:
df_users.describe(include='all')

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
count,12000.00000,12000,12000,12000,12000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000,8823.000000
unique,NaN,11996,11355,11980,5,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2014-02-11 17:57:53,Araujo Gabriela,AlfieLane@yahoo.com,ORG_INVITE,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,5,2,4254,NaN,NaN,NaN,NaN,NaN,NaN
mean,6000.50000,NaN,NaN,NaN,NaN,1.379279e+09,0.249500,0.149333,141.884583,5962.957145,0.181571
std,3464.24595,NaN,NaN,NaN,NaN,1.953116e+07,0.432742,0.356432,124.056723,3383.761968,0.385512
min,1.00000,NaN,NaN,NaN,NaN,1.338452e+09,0.000000,0.000000,0.000000,3.000000,0.000000
25%,3000.75000,NaN,NaN,NaN,NaN,1.363195e+09,0.000000,0.000000,29.000000,3058.000000,0.000000
50%,6000.50000,NaN,NaN,NaN,NaN,1.382888e+09,0.000000,0.000000,108.000000,5954.000000,0.000000
75%,9000.25000,NaN,NaN,NaN,NaN,1.398443e+09,0.000000,0.000000,238.250000,8817.000000,0.000000


In [17]:
df_users.isnull().sum()

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
adopted_user                  3177
dtype: int64

In [18]:
# if a user does not visit it means that the user did not count more than 3 times in 7 days. 
# So we will fill null with zero
df_users['adopted_user'].fillna(0, inplace=True)

In [19]:
# we can fill last_session_creation_time with 0 we assume those users are not adopted  users
df_users['last_session_creation_time'].fillna(0, inplace=True)

In [20]:
# checking for creation_source for null invited_by_user_ column
df_users[df_users.invited_by_user_id.isnull()].creation_source.unique()


array(['SIGNUP', 'PERSONAL_PROJECTS', 'SIGNUP_GOOGLE_AUTH'], dtype=object)

In [21]:
# Since they are not invited by any users, then we will fill missing values with zero
df_users['invited_by_user_id'].fillna(0,inplace=True)

In [22]:
df_users.describe(include='all')

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
count,12000.00000,12000,12000,12000,12000,1.200000e+04,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
unique,NaN,11996,11355,11980,5,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2014-02-11 17:57:53,Araujo Gabriela,AlfieLane@yahoo.com,ORG_INVITE,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,5,2,4254,NaN,NaN,NaN,NaN,NaN,NaN
mean,6000.50000,NaN,NaN,NaN,NaN,1.014115e+09,0.249500,0.149333,141.884583,3188.691333,0.133500
std,3464.24595,NaN,NaN,NaN,NaN,6.087938e+08,0.432742,0.356432,124.056723,3869.027693,0.340128
min,1.00000,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3000.75000,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,29.000000,0.000000,0.000000
50%,6000.50000,NaN,NaN,NaN,NaN,1.368880e+09,0.000000,0.000000,108.000000,875.000000,0.000000
75%,9000.25000,NaN,NaN,NaN,NaN,1.393729e+09,0.000000,0.000000,238.250000,6317.000000,0.000000


In [23]:
# convert creation_time to datetime data
df_users['creation_time']=pd.to_datetime(df_users['creation_time'])


In [24]:
# we will get a new column that shows how old the account is. 
df_users['days_since_creation'] = (df_users.creation_time.max()-df_users.creation_time).dt.days

In [25]:
df_users.drop(['creation_time',  'last_session_creation_time'], axis=1, inplace=True)

In [26]:
df_users.describe(include='all')

,object_id,name,email,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,days_since_creation
count,12000.00000,12000,12000,12000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
unique,NaN,11355,11980,5,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Araujo Gabriela,AlfieLane@yahoo.com,ORG_INVITE,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5,2,4254,NaN,NaN,NaN,NaN,NaN,NaN
mean,6000.50000,NaN,NaN,NaN,0.249500,0.149333,141.884583,3188.691333,0.133500,317.942917
std,3464.24595,NaN,NaN,NaN,0.432742,0.356432,124.056723,3869.027693,0.340128,216.641423
min,1.00000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3000.75000,NaN,NaN,NaN,0.000000,0.000000,29.000000,0.000000,0.000000,122.000000
50%,6000.50000,NaN,NaN,NaN,0.000000,0.000000,108.000000,875.000000,0.000000,298.000000
75%,9000.25000,NaN,NaN,NaN,0.000000,0.000000,238.250000,6317.000000,0.000000,500.000000


In [27]:
# using tfidf to tokenize emails domain. 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,3))

In [28]:
tfidf_email =tfidf.fit_transform(df_users['email'])

In [29]:
# droping email and names columns
df_users.drop(['email', 'name'], axis=1, inplace=True)
df_users.head()

,object_id,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,days_since_creation
0,1,GUEST_INVITE,1,0,11,10803.0,0.0,38
1,2,ORG_INVITE,0,0,1,316.0,1.0,196
2,3,ORG_INVITE,0,0,94,1525.0,0.0,437
3,4,GUEST_INVITE,0,0,1,5151.0,0.0,374
4,5,GUEST_INVITE,0,0,193,5240.0,0.0,498


In [30]:
df_users=pd.get_dummies(df_users, columns=['creation_source'])

In [31]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   object_id                           12000 non-null  int64  
 1   opted_in_to_mailing_list            12000 non-null  int64  
 2   enabled_for_marketing_drip          12000 non-null  int64  
 3   org_id                              12000 non-null  int64  
 4   invited_by_user_id                  12000 non-null  float64
 5   adopted_user                        12000 non-null  float64
 6   days_since_creation                 12000 non-null  int64  
 7   creation_source_GUEST_INVITE        12000 non-null  uint8  
 8   creation_source_ORG_INVITE          12000 non-null  uint8  
 9   creation_source_PERSONAL_PROJECTS   12000 non-null  uint8  
 10  creation_source_SIGNUP              12000 non-null  uint8  
 11  creation_source_SIGNUP_GOOGLE_AUTH  12000

In [32]:
df1 = pd.DataFrame(tfidf_email.toarray(), columns=tfidf.get_feature_names())


/Users/bilalturan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
# combine tfidf  vectorizer df and df_users
df_final = pd.concat([df_users,df1],axis=1)
df_final.head()

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,days_since_creation,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,aaliyaharmstrong,aaliyaharmstrong yahoo,aaliyaharmstrong yahoo com,aaliyahgibbs,aaliyahgibbs gustr,aaliyahgibbs gustr com,aaliyahheath,aaliyahheath jourrapide,aaliyahheath jourrapide com,aaliyahiqbal,aaliyahiqbal gustr,aaliyahiqbal gustr com,aaliyahmarshall,aaliyahmarshall yahoo,aaliyahmarshall yahoo com,aaliyahpatterson,aaliyahpatterson jourrapide,aaliyahpatterson jourrapide com,aaliyahpreston,aaliyahpreston jourrapide,aaliyahpreston jourrapide com,aaliyahstevenson,aaliyahstevenson cuvox,aaliyahstevenson cuvox de,aaliyahtomlinson,aaliyahtomlinson yahoo,aaliyahtomlinson yahoo com,aaronatkins,aaronatkins jourrapide,aaronatkins jourrapide com,aaronbond,aaronbond jourrapide,aaronbond jourrapide com,aaronbowen,aaronbowen gmail,aaronbowen gmail com,aaronchan,aaronchan gustr,aaronchan gustr com,aaronchandler,aaronchandler gmail,aaronchandler gmail com,aarondrew,aarondrew jourrapide,aarondrew jourrapide com,aaronfitzgerald,aaronfitzgerald gmail,aaronfitzgerald gmail com,aaronfitzsimmons,aaronfitzsimmons yahoo,aaronfitzsimmons yahoo com,aaronfrench,aaronfrench cuvox,aaronfrench cuvox de,aarongirdlestone,aarongirdlestone gmail,aarongirdlestone gmail com,aarongottschalk,aarongottschalk gmail,aarongottschalk gmail com,aarongraham,aarongraham gmail,aarongraham gmail com,aaronhammond,aaronhammond hotmail,aaronhammond hotmail com,aaronholroyd,aaronholroyd hotmail,aaronholroyd hotmail com,aaronhowell,aaronhowell gmail,aaronhowell gmail com,aaronhussain,aaronhussain hotmail,aaronhussain hotmail com,aaronhutchinson,aaronhutchinson jourrapide,aaronhutchinson jourrapide com,aaronlittle,aaronlittle gmail,aaronlittle gmail com,aaronmaughan,aaronmaughan yahoo,aaronmaughan yahoo com,aaronrmorse,aaronrmorse jourrapide,aaronrmorse jourrapide com,aaronroper,aaronroper gmail,aaronroper gmail com,aaronschey,aaronschey gmail,aaronschey gmail com,aaronsmanders,aaronsmanders yahoo,aaronsmanders yahoo com,aaronspencer,aaronspencer cuvox,aaronspencer cuvox de,aaronswift,aaronswift gmail,aaronswift gmail com,aarontipper,aarontipper yahoo,aarontipper yahoo com,aaronwhittaker,aaronwhittaker hotmail,aaronwhittaker hotmail com,aasedtoft,aasedtoft yahoo,aasedtoft yahoo com,aasefhenriksen,aasefhenriksen gmail,aasefhenriksen gmail com,aasehjrgensen,aasehjrgensen gustr,aasehjrgensen gustr com,aasejbruun,aasejbruun yahoo,aasejbruun yahoo com,aasemfrandsen,aasemfrandsen jourrapide,aasemfrandsen jourrapide com,aaseothygesen,aaseothygesen jourrapide,aaseothygesen jourrapide com,aasepjuhl,aasepjuhl jourrapide,aasepjuhl jourrapide com,aasesfrandsen,aasesfrandsen gmail,aasesfrandsen gmail com,abbeygerrald,abbeygerrald jourrapide,abbeygerrald jourrapide com,abbeygilbert,abbeygilbert jourrapide,abbeygilbert jourrapide com,abbeyhammond,abbeyhammond jourrapide,abbeyhammond jourrapide com,abbeyholtze,abbeyholtze jourrapide,abbeyholtze jourrapide com,abbeykinnear,abbeykinnear jourrapide,abbeykinnear jourrapide com,abbeyrowlandson,abbeyrowlandson yahoo,abbeyrowlandson yahoo com,abbeysadler,abbeysadler jourrapide,abbeysadler jourrapide com,abbeystout,abbeystout gmail,abbeystout gmail com,abbeytraeger,abbeytraeger gustr,abbeytraeger gustr com,abbeywinneke,abbeywinneke gmail,abbeywinneke gmail com,abbeyzimpel,abbeyzimpel hotmail,abbeyzimpel hotmail com,abbiearmstrong,abbiearmstrong yahoo,abbiearmstrong yahoo com,abbiecarpenter,abbiecarpenter yahoo,abbiecarpenter yahoo com,abbieconnolly,abbieconnolly gustr,abbieconnolly gustr com,abbiefuller,abbiefuller yahoo,abbiefuller yahoo com,abbiegardner,abbiegardner gmail,abbiegardner gmail com,abbiehoward,abbiehoward yahoo,abbiehoward yahoo com,abbiekennedy,abbiekennedy yahoo,abbiekennedy yahoo com,abbiematthews,abbiematthews gmail,abbiematthews gmail com,abbienorris,abbienorris 

### Modeling

In [34]:
X = df_final.loc[:, df_final.columns!='adopted_user']
y = df_final.adopted_user

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=3)
# We want to check the shape of the X_train, y_train, X_test and y_test to make sure the proportions are right.
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

X_train:  (8400, 38339)
y_train:  (8400,)
X_test:  (3600, 38339)
y_test:  (3600,)


In [37]:
# we have imbalance data
# check the distribution
df_final['adopted_user'].value_counts()/df_final.shape[0]



0.0    0.8665
1.0    0.1335
Name: adopted_user, dtype: float64

#### 1. model

In [ ]:
lr = LogisticRegression(random_state=123, class_weight={0:0.13, 1:0.87})
lr.fit(X_train,y_train)

In [ ]:
print(classification_report(y_test,lr.predict(X_test)))

We use logistic regression. we have 0.42 accuracy in this model

In [ ]:
X = df_users.loc[:, df_users.columns !='adopted_user']
y =df_users.adopted_user

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X ,y, test_size=0.3, random_state=32)
# We want to check the shape of the X_train, y_train, X_test and y_test to make sure the proportions are right.
print('X_train2: ', X_train2.shape)
print('y_train2: ', y_train2.shape)
print('X_test2: ', X_test2.shape)
print('y_test2: ', y_test2.shape)

#### 2. model

In [ ]:
lr2 = LogisticRegression(random_state=12, class_weight={0:0.13, 1:0.87})
lr2.fit(X_train2,y_train2)

In [ ]:
print(classification_report(y_test2,lr2.predict(X_test2)))

We got 0.39 accuracy in the last model. 